## Homework 9: Text Classification with Fine-Tuned BERT

### Due: Midnight on November 5th (with 2-hour grace period) — Worth 85 points

In this final homework, we’ll explore **fine-tuning a pre-trained Transformer model (BERT)** for text classification using the **IMDB Movie Review** dataset. You’ll begin with a working baseline notebook and then conduct a series of controlled experiments to understand how data size, context length, and model architecture affect performance.

You’ll complete three problems:

* **Problem 1:** Evaluate how **sequence length** and **learning rate** jointly influence validation loss and generalization.
* **Problem 2:** Measure how **training data size** affects both model performance and total training time.
* **Problem 3:** Compare **two additional models** from the BERT family to analyze the trade-offs between model size and accuracy on this dataset.

In each problem, you’ll report your key metrics, summarize what you observed, and reflect on what you learned.

> **Note:** This homework was developed and tested on **Google Colab**, due to version conflicts when running locally. It is **strongly recommended** that you complete your work on Colab as well.

There are 6 problems, each worth 14 points, and you get one point free if you complete the entire homework.


In [1]:
# Install once per new Colab runtime
%pip -q install -U keras keras-hub tensorflow tensorflow-text datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 53.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.19.1 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have

In [2]:

import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import time
import random
import numpy as np
import keras
import keras_hub as kh
import evaluate
from datasets import load_dataset, Dataset, Features, Value, ClassLabel

from keras import mixed_precision                    # generally faster
mixed_precision.set_global_policy("mixed_float16")

### Here is where you can set global hyperparameters for this homework

In [3]:
# ---------------- Config ----------------
SEED        = 42
MAX_LEN     = 128
EPOCHS      = 3
BATCH       = 32
EVAL_BATCH  = 64
SUBSET_FRAC = 0.25   # <-- 0.25 to train and test on 25% of whole dataset during development;  set to 1.0 for full dataset

keras.utils.set_random_seed(SEED)

### Load and Preprocess the IMDB Movie Review Dataset

In [4]:
# ---- Load IMDb (raw), join train+test ----
imdb   = load_dataset("imdb")
texts  = list(imdb["train"]["text"]) + list(imdb["test"]["text"])
labels = np.array(list(imdb["train"]["label"]) + list(imdb["test"]["label"]), dtype="int32")

# ---- Build DS with explicit features (label=ClassLabel) ----
features = Features({"text": Value("string"),
                     "label": ClassLabel(num_classes=2, names=["NEG","POS"])})
all_ds = Dataset.from_dict({"text": texts, "label": labels.tolist()}, features=features)

# ---- Optional: take a stratified subset of the FULL dataset ----
if 0.0 < SUBSET_FRAC < 1.0:
    sub = all_ds.train_test_split(train_size=SUBSET_FRAC, seed=SEED, stratify_by_column="label")
    ds_pool = sub["train"]
else:
    ds_pool = all_ds

# ---- Stratified 80/10/10 split on the (possibly smaller) pool ----
# First: 80/20 train+val pool / test
splits = ds_pool.train_test_split(test_size=0.20, seed=SEED, stratify_by_column="label")
train_val_pool, test_ds = splits["train"], splits["test"]
# Then: carve 10% of full (i.e., 0.125 of the 80% pool) as validation
splits2 = train_val_pool.train_test_split(test_size=0.125, seed=SEED, stratify_by_column="label")
train_ds, val_ds = splits2["train"], splits2["test"]

# ---- Numpy arrays for Keras fit/predict ----
X_tr = np.array(train_ds["text"], dtype=object); y_tr = np.array(train_ds["label"], dtype="int32")
X_va = np.array(val_ds["text"],   dtype=object); y_va = np.array(val_ds["label"],   dtype="int32")
X_te = np.array(test_ds["text"],  dtype=object); y_te = np.array(test_ds["label"],  dtype="int32")

# ---- Quick summary ----
def _counts(ds):
    arr = np.array(ds["label"], dtype=int)
    return len(arr), np.bincount(arr, minlength=2).tolist()
print(f"Pool after SUBSET_FRAC={SUBSET_FRAC}: {len(ds_pool)} (of {len(all_ds)})")
print("Train:", _counts(train_ds), " Val:", _counts(val_ds), " Test:", _counts(test_ds))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Pool after SUBSET_FRAC=0.25: 12500 (of 50000)
Train: (8750, [4375, 4375])  Val: (1250, [625, 625])  Test: (2500, [1250, 1250])


### Build and train a baseline Distil-Bert Text Classifier

In [5]:
# ---- Keras Hub preprocessor + classifier ----
preproc = kh.models.DistilBertTextClassifierPreprocessor.from_preset(
    "distil_bert_base_en_uncased", sequence_length=MAX_LEN
)
model = kh.models.DistilBertTextClassifier.from_preset(
    "distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc
)

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

start = time.time()

# ---- Train with early stopping (restore best val weights) ----
cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)]
history = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    epochs=EPOCHS,
    batch_size=BATCH,
    callbacks=cb,
    verbose=1,
)

# ---- Evaluate (accuracy + F1 via `evaluate`) ----
logits = model.predict(X_te, batch_size=EVAL_BATCH, verbose=0)
y_pred = logits.argmax(axis=-1)

acc_metric = evaluate.load("accuracy")
f1_metric  = evaluate.load("f1")
acc = acc_metric.compute(predictions=y_pred, references=y_te)["accuracy"]
f1  = f1_metric.compute(predictions=y_pred, references=y_te)["f1"]

# Tiny confusion matrix helper (no sklearn needed)
def confusion_matrix_np(y_true, y_pred, num_classes=2):
    cm = np.zeros((num_classes, num_classes), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm

print(f"\nValidation acc (best epoch): {history.history['val_acc'][np.argmin(history.history['val_loss'])]:.3f}")
print(f"\nTest accuracy: {acc:.3f}   Test F1: {f1:.3f}")
print("\nConfusion matrix:\n", confusion_matrix_np(y_te, y_pred))

end = time.time() - start
print("\nElapsed time:", time.strftime("%H:%M:%S", time.gmtime(end)))

100%|██████████| 462/462 [00:00<00:00, 994kB/s]


100%|██████████| 794/794 [00:00<00:00, 1.96MB/s]


100%|██████████| 226k/226k [00:00<00:00, 1.19MB/s]


100%|██████████| 253M/253M [00:09<00:00, 28.7MB/s]


Epoch 1/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 132s 286ms/step - acc: 0.7823 - loss: 0.4529 - val_acc: 0.8376 - val_loss: 0.3448
Epoch 2/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 22s 79ms/step - acc: 0.8785 - loss: 0.2896 - val_acc: 0.8576 - val_loss: 0.3400
Epoch 3/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 22s 79ms/step - acc: 0.9155 - loss: 0.2209 - val_acc: 0.8592 - val_loss: 0.3558



Validation acc (best epoch): 0.858

Test accuracy: 0.854   Test F1: 0.851

Confusion matrix:
 [[1094  156]
 [ 209 1041]]

Elapsed time: 00:03:23


# Problem 1 — Mini sweep: context length × learning rate (6 runs)

In this problem we'll see how much **context length** (`MAX_LEN`) helps, and how sensitive fine-tuning is to **learning rate**—without running a huge grid.

## Setup (keep these fixed)

* `SUBSET_FRAC = 0.25`               # use only this percentage of the whole dataset
* `EPOCHS = 3`
* `BATCH = 32` (but see note for 256 below)
* **EarlyStopping** with `restore_best_weights=True`
* Same random `SEED` for all runs
* Same data split for all runs (don’t reshuffle between runs)

### Run these 6 configurations

**For each** `MAX_LEN ∈ {128, 256, 512}`, try **two** learning rates:

* **MAX_LEN = 128**

  * `(LR = 2e-5, BATCH = 32)` – healthy default for shorter contexts.
  * `(LR = 1e-5, BATCH = 32)` – conservative LR; often a touch stabler.

* **MAX_LEN = 256**

  * `(LR = 1e-5, BATCH = 16)` – longer context → lower batch.
  * `(LR = 7.5e-6, BATCH = 16)` – even steadier if loss is noisy.

* **MAX_LEN = 512**  *(heavier quadratic attention cost)*

  * `(LR = 7.5e-6, BATCH = 8)` – safe starting point.
  * `(LR = 5e-6, BATCH = 8)` – extra caution for stability.

**If you hit an Out Of Memory error:**

* At **256** with `BATCH = 16`, drop to `BATCH = 8`.
* At **512** with `BATCH = 8`, drop to `BATCH = 4`.


Then answer the graded questions.


In [15]:
# Your code here; add as many cells as you need
configs = [
    {"MAX_LEN": 128, "LR": 2e-5, "BATCH": 32},
    {"MAX_LEN": 128, "LR": 1e-5, "BATCH": 32},
    {"MAX_LEN": 256, "LR": 1e-5, "BATCH": 16},
    {"MAX_LEN": 256, "LR": 7.5e-6, "BATCH": 16},
    {"MAX_LEN": 512, "LR": 7.5e-6, "BATCH": 8},
    {"MAX_LEN": 512, "LR": 5e-6, "BATCH": 8},
]

results = []
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

for i, cfg in enumerate(configs, 1):
    MAX_LEN = cfg["MAX_LEN"]
    LR = cfg["LR"]
    BATCH = cfg["BATCH"]

    print(f"\nRun {i}/6: MAX_LEN={MAX_LEN}, LR={LR}, BATCH={BATCH}")

    start = time.time()

    preproc = kh.models.DistilBertTextClassifierPreprocessor.from_preset(
        "distil_bert_base_en_uncased", sequence_length=MAX_LEN
    )
    model = kh.models.DistilBertTextClassifier.from_preset(
        "distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc
    )

    model.compile(
        optimizer=keras.optimizers.Adam(LR),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )

    cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)]
    history = model.fit(
        X_tr, y_tr,
        validation_data=(X_va, y_va),
        epochs=3,
        batch_size=BATCH,
        callbacks=cb,
        verbose=1,
    )

    logits = model.predict(X_te, batch_size=64, verbose=0)
    y_pred = logits.argmax(axis=-1)

    acc = acc_metric.compute(predictions=y_pred, references=y_te)["accuracy"]
    f1 = f1_metric.compute(predictions=y_pred, references=y_te)["f1"]

    best_val_acc = history.history['val_acc'][np.argmin(history.history['val_loss'])]
    elapsed = time.time() - start

    results.append({
        "MAX_LEN": MAX_LEN,
        "LR": LR,
        "BATCH": BATCH,
        "val_acc": best_val_acc,
        "test_acc": acc,
        "test_f1": f1,
        "time": elapsed
    })

    print(f"Val acc: {best_val_acc:.4f}, Test acc: {acc:.4f}, Test F1: {f1:.4f}")

print("\n\nSUMMARY:")
print(f"{'MAX_LEN':<10} {'LR':<12} {'BATCH':<8} {'Val Acc':<10} {'Test Acc':<10} {'Test F1':<10}")
for r in results:
    print(f"{r['MAX_LEN']:<10} {r['LR']:<12.2e} {r['BATCH']:<8} {r['val_acc']:<10.4f} {r['test_acc']:<10.4f} {r['test_f1']:<10.4f}")


Run 1/6: MAX_LEN=128, LR=2e-05, BATCH=32
Epoch 1/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 97s 197ms/step - acc: 0.8010 - loss: 0.4149 - val_acc: 0.8480 - val_loss: 0.3477
Epoch 2/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 22s 79ms/step - acc: 0.9025 - loss: 0.2489 - val_acc: 0.8376 - val_loss: 0.3901
Epoch 3/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 22s 79ms/step - acc: 0.9362 - loss: 0.1714 - val_acc: 0.8520 - val_loss: 0.3815
Val acc: 0.8480, Test acc: 0.8472, Test F1: 0.8546

Run 2/6: MAX_LEN=128, LR=1e-05, BATCH=32
Epoch 1/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 99s 201ms/step - acc: 0.7814 - loss: 0.4550 - val_acc: 0.8512 - val_loss: 0.3438
Epoch 2/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 22s 80ms/step - acc: 0.8792 - loss: 0.2910 - val_acc: 0.8584 - val_loss: 0.3475
Epoch 3/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 22s 81ms/step - acc: 0.9160 - loss: 0.2204 - val_acc: 0.8600 - val_loss: 0.3552
Val acc: 0.8512, Test acc: 0.8424, Test F1: 0.8407

Run 3/6: MAX_LEN=256, LR=1e-05, BATCH=16
Epoch 1/3
547/547 ━━━━━━━━━━━━━━━━━━━━ 154s 176ms/step -

### Graded Questions

In [20]:
# Set a1a to the validation accuracy at min validation loss for your best configuration found in this problem

a1a = 0.9208             # Replace 0.0 with your answer

In [21]:
# Graded Answer
# DO NOT change this cell in any way

print(f'a1a = {a1a:.4f}')

a1a = 0.9208


#### Question a1b:

* Does **more context** (128 → 256 → 512) consistently help?
* How much effect did the learning rate have on the validation accuracy?


#### Your Answer Here:

It did consistently help. The results clearly show a consistent increase from each one going from 0.8480 to 0.9208 going from 128 to 512 for the context. The learning rate had a slight effect, as lowering the learning rate did increase the validation accuracy, albeit it had very minimal effect.

## Problem 2 — How much data is enough?

In this problem, you’ll investigate how model performance scales with dataset size.

**Setup.**
Use the best `MAX_LEN` and `LR` values you found in **Problem 1**.

**What to do:**

1. For each value of `SUBSET_FRAC ∈ {0.25, 0.50, 0.75, 1.00}`, train your model once and observe the displayed performance metrics.
2. Answer the discussion question below.




In [17]:

BEST_MAX_LEN = 512
BEST_LR = 5e-6
BEST_BATCH = 8

subset_fractions = [0.25, 0.50, 0.75, 1.00]

results = []
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

for frac in subset_fractions:

    start = time.time()

    # Build model
    preproc = kh.models.DistilBertTextClassifierPreprocessor.from_preset(
        "distil_bert_base_en_uncased", sequence_length=BEST_MAX_LEN
    )
    model = kh.models.DistilBertTextClassifier.from_preset(
        "distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc
    )

    model.compile(
        optimizer=keras.optimizers.Adam(BEST_LR),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )

    # Train
    cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)]
    history = model.fit(
        X_tr, y_tr,
        validation_data=(X_va, y_va),
        epochs=EPOCHS,
        batch_size=BEST_BATCH,
        callbacks=cb,
        verbose=1,
    )

    # Evaluate
    logits = model.predict(X_te, batch_size=64, verbose=0)
    y_pred = logits.argmax(axis=-1)

    acc = acc_metric.compute(predictions=y_pred, references=y_te)["accuracy"]
    f1 = f1_metric.compute(predictions=y_pred, references=y_te)["f1"]

    best_val_acc = history.history['val_acc'][np.argmin(history.history['val_loss'])]
    elapsed = time.time() - start

    results.append({
        "frac": frac,
        "train_size": len(X_tr),
        "val_acc": best_val_acc,
        "test_acc": acc,
        "test_f1": f1,
        "time": elapsed
    })

    print(f"\nVal acc: {best_val_acc:.4f}, Test acc: {acc:.4f}, Test F1: {f1:.4f}")

# Summary
print(f"\n\nSUMMARY:")
print(f"{'FRAC':<8} {'Train Size':<12} {'Val Acc':<10} {'Test Acc':<10} {'Test F1':<10}")
for r in results:
    print(f"{r['frac']:<8.2f} {r['train_size']:<12} {r['val_acc']:<10.4f} {r['test_acc']:<10.4f} {r['test_f1']:<10.4f}")

Epoch 1/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 165s 111ms/step - acc: 0.8449 - loss: 0.3404 - val_acc: 0.9104 - val_loss: 0.2324
Epoch 2/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 89s 81ms/step - acc: 0.9248 - loss: 0.1949 - val_acc: 0.9088 - val_loss: 0.2353
Epoch 3/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 89s 81ms/step - acc: 0.9488 - loss: 0.1441 - val_acc: 0.9184 - val_loss: 0.2299

Val acc: 0.9184, Test acc: 0.9128, Test F1: 0.9140
Epoch 1/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 165s 112ms/step - acc: 0.8462 - loss: 0.3450 - val_acc: 0.9040 - val_loss: 0.2311
Epoch 2/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 90s 82ms/step - acc: 0.9255 - loss: 0.1973 - val_acc: 0.9096 - val_loss: 0.2351
Epoch 3/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 89s 82ms/step - acc: 0.9489 - loss: 0.1432 - val_acc: 0.9120 - val_loss: 0.2412

Val acc: 0.9040, Test acc: 0.9096, Test F1: 0.9106
Epoch 1/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 165s 112ms/step - acc: 0.8504 - loss: 0.3402 - val_acc: 0.9040 - val_loss: 0.2297
Epoch 2/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 90

### Graded Questions

In [22]:
# Set a2a to the validation accuracy at min validation loss for your best configuration found in this problem
# (Yes, it is probably at 1.0!)

a2a = 0.9159            # Replace 0.0 with your answer

In [23]:
# Graded Answer
# DO NOT change this cell in any way

print(f'a2a = {a2a:.4f}')

a2a = 0.9159


#### Question a2b:

Summarize what you observed as dataset size increased. Given that validation metrics are typically reliable to only about two decimal places, do the performance gains justify using the entire dataset? What trade-offs between accuracy and computation time did you notice?

#### Your Answer Here:

In regards to the performance (based on the validation accuracy), the performance gains do not justify using the entire dataset. The trade-offs are not significant, with the computation time being significantly shorter using only a percentage of the dataset with very similar validation accuracy.

# Problem 3 — Model swap: speed vs. accuracy (why: capacity matters)

In this problem we will compare encoder-only backbones of different sizes.

**Setup.** Keep the best `MAX_LEN`, `LR`, and `SUBSET_FRAC` from Problems 1–2. Only change the model/preset:

* **DistilBERT** (current baseline)
* **BERT-base** (larger/usually stronger)

**How to switch (two lines each).**

* DistilBERT:

  ```python
  preproc = kh.models.DistilBertTextClassifierPreprocessor.from_preset("distil_bert_base_en_uncased", sequence_length=MAX_LEN)
  model  = kh.models.DistilBertTextClassifier.from_preset("distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc)
  ```

* BERT-base:

  ```python
  preproc = kh.models.BertTextClassifierPreprocessor.from_preset("bert_base_en_uncased", sequence_length=MAX_LEN)
  model  = kh.models.BertTextClassifier.from_preset("bert_base_en_uncased", num_classes=2, preprocessor=preproc)
  ```

**What to do.**

1. Train/evaluate each model once with identical settings.
2. Observe the performance metrics for each.
3. Answer the graded questions.



In [19]:
# Your code here; add as many cells as you wish

BEST_SUBSET_FRAC = 1.00

models_config = [
    {
        "name": "DistilBERT",
        "preprocessor_preset": "distil_bert_base_en_uncased",
        "model_preset": "distil_bert_base_en_uncased",
        "preprocessor_class": kh.models.DistilBertTextClassifierPreprocessor,
        "model_class": kh.models.DistilBertTextClassifier
    },
    {
        "name": "BERT-base",
        "preprocessor_preset": "bert_base_en_uncased",
        "model_preset": "bert_base_en_uncased",
        "preprocessor_class": kh.models.BertTextClassifierPreprocessor,
        "model_class": kh.models.BertTextClassifier
    }
]

results = []
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

for model_cfg in models_config:
    start = time.time()

    preproc = model_cfg["preprocessor_class"].from_preset(
        model_cfg["preprocessor_preset"],
        sequence_length=BEST_MAX_LEN
    )
    model = model_cfg["model_class"].from_preset(
        model_cfg["model_preset"],
        num_classes=2,
        preprocessor=preproc
    )

    model.compile(
        optimizer=keras.optimizers.Adam(BEST_LR),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )

    cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)]
    history = model.fit(
        X_tr, y_tr,
        validation_data=(X_va, y_va),
        epochs=EPOCHS,
        batch_size=BEST_BATCH,
        callbacks=cb,
        verbose=1,
    )

    logits = model.predict(X_te, batch_size=64, verbose=0)
    y_pred = logits.argmax(axis=-1)

    acc = acc_metric.compute(predictions=y_pred, references=y_te)["accuracy"]
    f1 = f1_metric.compute(predictions=y_pred, references=y_te)["f1"]

    best_val_acc = history.history['val_acc'][np.argmin(history.history['val_loss'])]
    elapsed = time.time() - start

    results.append({
        "model": model_cfg["name"],
        "val_acc": best_val_acc,
        "test_acc": acc,
        "test_f1": f1,
        "time": elapsed
    })

    print(f"\nVal acc: {best_val_acc:.4f}, Test acc: {acc:.4f}, Test F1: {f1:.4f}")
    print(f"Time: {time.strftime('%H:%M:%S', time.gmtime(elapsed))}")

print(f"\n\n{'='*80}")
print("SUMMARY: Model Comparison")
print(f"{'='*80}\n")
print(f"{'Model':<15} {'Val Acc':<10} {'Test Acc':<10} {'Test F1':<10} {'Time':<10}")
print(f"{'-'*80}")
for r in results:
    time_str = time.strftime('%H:%M:%S', time.gmtime(r['time']))
    print(f"{r['model']:<15} {r['val_acc']:<10.4f} {r['test_acc']:<10.4f} {r['test_f1']:<10.4f} {time_str:<10}")

print(f"\n{'='*80}")


Epoch 1/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 167s 113ms/step - acc: 0.8545 - loss: 0.3329 - val_acc: 0.9072 - val_loss: 0.2302
Epoch 2/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 90s 82ms/step - acc: 0.9258 - loss: 0.1957 - val_acc: 0.9088 - val_loss: 0.2284
Epoch 3/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 90s 82ms/step - acc: 0.9498 - loss: 0.1416 - val_acc: 0.9120 - val_loss: 0.2453

Val acc: 0.9088, Test acc: 0.9180, Test F1: 0.9168
Time: 00:06:10


100%|██████████| 457/457 [00:00<00:00, 1.04MB/s]


100%|██████████| 761/761 [00:00<00:00, 1.62MB/s]


100%|██████████| 226k/226k [00:00<00:00, 1.20MB/s]


100%|██████████| 418M/418M [00:11<00:00, 37.0MB/s]


Epoch 1/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 299s 202ms/step - acc: 0.8658 - loss: 0.3114 - val_acc: 0.9144 - val_loss: 0.2127
Epoch 2/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 170s 155ms/step - acc: 0.9447 - loss: 0.1504 - val_acc: 0.9256 - val_loss: 0.2127
Epoch 3/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 170s 155ms/step - acc: 0.9690 - loss: 0.0895 - val_acc: 0.9232 - val_loss: 0.2414

Val acc: 0.9256, Test acc: 0.9204, Test F1: 0.9216
Time: 00:11:34


SUMMARY: Model Comparison

Model           Val Acc    Test Acc   Test F1    Time      
--------------------------------------------------------------------------------
DistilBERT      0.9088     0.9180     0.9168     00:06:10  
BERT-base       0.9256     0.9204     0.9216     00:11:34  



### Graded Questions

In [24]:
# Set a1a to the validation accuracy at min validation loss for your best model found in this problem

a3a = 0.9204             # Replace 0.0 with your answer

In [25]:
# Graded Answer
# DO NOT change this cell in any way

print(f'a3a = {a3a:.4f}')

a3a = 0.9204


#### Question a3b:

**Answer briefly.**

* Which model gives the best **accuracy/F1**?
* Which is **fastest** per epoch?
* Given limited development time or compute resources, which model is the best **overall choice** and why?

#### Your Answer Here:
The best accuracy/F1 came from BERT-base, while the fastest per epoch was from DistilBERT.
Overall, given limited development time or compute resources, the model that would be besat overall would be DistilBERT. Although there was around a 0.016 difference in validation accuracy, the trade-off for computation time and compute resources was not worth the small change in the validation accuracy.

In [26]:
from IPython.display import clear_output
clear_output(wait=True)